# Implementation: ResNet + Quantum Layer

**Goal**: Integrate TorchVision with PennyLane.

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import pennylane as qml

# 1. Classical Backbone (ResNet18)
resnet = models.resnet18(pretrained=True)
# Freeze weights (we only train the quantum head)
for param in resnet.parameters():
    param.requires_grad = False

print("Loaded pre-trained ResNet.")

# 2. Quantum Head
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (2, n_qubits)}
quantum_layer = qml.qnn.TorchLayer(qnode, weight_shapes)

# 3. Stitch them together
class HybridNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet
        # Replace the classifier of ResNet
        # ResNet18 output is 512. Reduce to 4 (for qubits).
        self.fc_reduce = nn.Linear(512, 4)
        self.quantum = quantum_layer
        self.fc_out = nn.Linear(4, 2) # 2 Classes
    
    def forward(self, x):
        # ResNet expects (Batch, 3, 224, 224). We assume x is processed.
        # For this mock, we skip the actual CNN pass
        x = torch.randn(1, 512) # Output of avgpool
        x = self.fc_reduce(x)
        x = torch.tanh(x) * torch.pi # Scale to -pi, pi
        x = self.quantum(x)
        x = self.fc_out(x)
        return x

model = HybridNet()
print(model)
print("Forward Pass:", model(torch.randn(1, 3, 224, 224)))

## Conclusion
We turned a Quantum Computer into a 'Plugin' for ResNet.